In [1]:
# import necessary libs
import torch
import pickle
import numpy as np
from dis import dis
from astyx_utils import *
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Define constants
DATA_DIR = "./data/"
IMAGE_DIR = DATA_DIR + "image/"
PCD_DIR = DATA_DIR + "pcd/"
BOX2D_DIR = DATA_DIR + "box2d/"
BOX3D_DIR = DATA_DIR + "box3d/"
PRED_DICT_2D_DIR = DATA_DIR + "pred_dict_2D/"
PRED_DICT_3D_DIR = DATA_DIR + "pred_dict_3D/"

In [3]:
# Getting filename for image, pcd, box2d and box3d
file_list_image=os.listdir(IMAGE_DIR)
file_list_no_ex = [x.split('.')[0] for x in file_list_image]
file_list_pcd = [x+".npy" for x in file_list_no_ex]
file_list_box2d = [x+"_box2d.pickle" for x in file_list_no_ex]
file_list_box3d = [x+"_box3d.pickle" for x in file_list_no_ex]

In [4]:
# Read images from camera
image_list = []
for path in file_list_image:
    img = plt.imread(IMAGE_DIR + path)
    image_list.append(img)
images = np.array(image_list)

In [5]:
# Read pcd from lidar
pcd_list = []
for path in file_list_pcd:
    pcd = np.load(PCD_DIR + path)
    pcd_list.append(pcd)
pcds = np.array(pcd_list)

In [6]:
# Read 2d boxes from Detectron
# box2d_list = []
# for path in file_list_box2d:
#     with open(BOX2D_DIR + path, 'rb') as handle:
#         box2d = pickle.load(handle)
#         box2d_list.append(box2d.numpy())
# boxes2d = np.array(box2d_list)
# boxes2d

In [7]:
# Read 3d boxes center from OpenPCDet with pointpillar
# box3d_list = []
# for path in file_list_box3d:
#     with open(BOX3D_DIR + path, 'rb') as handle:
#         box3d = pickle.load(handle)
#         box3d_list.append(box3d.numpy()[:,:3])
# boxes3d = np.array(box3d_list)

In [8]:
# boxes3d_to_img = []
# for frame in boxes3d:
#     frame_list = lidar2CameraOurs(frame[:, [1, 2, 0]])
#     boxes3d_to_img.append(frame_list)

In [9]:
def detect_inlier(boxes2d, boxes3d):
    """
    Detect from the 3D boudning boexes if they are inside the 2D bounding boxes
    Args:
    boxes2d : np.array of shape [n, 4]
              boxes2d element [x1, y1, x2, y2]
              x1 -------- y1
               |          |
               |          |
               |          |
              x2 -------- y2
    boxes3d : np.array of shape [m, 2]
              boxes3d element [x, y, z]
    Return:
    list : [boolean] indicating if 3d boxes are inliers
    """
    result = []
    for box3d in boxes3d:
        inlier = False
        for box2d in boxes2d:
            if box3d[0] >= box2d[0] and box3d[0] <= box2d[2] and\
               box3d[1] >= box2d[1] and box3d[1] <= box2d[3]:
               inlier = True
        result.append(inlier)
    return result

In [10]:
# sum(detect_inlier(boxes2d[0], boxes3d_to_img[0]))

In [11]:
for file in file_list_no_ex:
    pred_dict_2D_path = PRED_DICT_2D_DIR + file + "_pred_dict_2D.pickle"
    pred_dict_3D_path = PRED_DICT_3D_DIR + file + "_pred_dict.pickle"
    with open(pred_dict_2D_path, 'rb') as handle:
        pred_dict_2D = pickle.load(handle)
        boxes_2D = pred_dict_2D["boxes"]
        scores_2D = pred_dict_2D["scores"]
        classes_2D = pred_dict_2D["classes"]
    with open(pred_dict_3D_path, 'rb') as handle:
        pred_dict_3D = pickle.load(handle)
        boxes_3D = pred_dict_3D["pred_boxes"]
        scores_3D = pred_dict_3D["pred_scores"]
        labels_3D = pred_dict_3D["pred_labels"]
    boxes_3D_filtered = []
    scores_3D_filtered = []
    labels_3D_filtered = []
    boxes_3D_to_img = lidar2CameraOurs(boxes_3D[:, [1, 2, 0]])
    result = detect_inlier(boxes_2D, boxes_3D_to_img)
    pred_dict_3D_filtered = {}
    for i in range(len(result)):
        if result[i]:
            boxes_3D_filtered.append(boxes_3D[i])
            scores_3D_filtered.append(scores_3D[i])
            labels_3D_filtered.append(labels_3D[i])
    pred_dict_3D_filtered["pred_boxes"] = boxes_3D_filtered
    pred_dict_3D_filtered["pred_scores"] = scores_3D_filtered
    pred_dict_3D_filtered["pred_labels"] = labels_3D_filtered
    output_file_name = "./outputs/" + file + "_pred_dict_filtered.pickle"
    with open(output_file_name, 'wb') as handle:
        pickle.dump(pred_dict_3D_filtered, handle)


In [12]:
print(pred_dict_3D_filtered)

{'pred_boxes': [tensor([40.7637,  8.7568,  0.2561,  3.9804,  1.5785,  1.4473,  5.7127]), tensor([50.4835, 14.3476,  1.2112,  4.0322,  1.6215,  1.5583,  5.6357])], 'pred_scores': [tensor(0.4060), tensor(0.2736)], 'pred_labels': [tensor(1), tensor(1)]}


In [ ]:
# experiment space
